In [1]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
ds = pd.read_csv('../input/starbucks.csv', encoding="utf-8")

In [4]:
ds.shape

(25600, 13)

Hacemos scraping de countrycode.org para traernos el nombre del país a partir del codigo ISO del dataset

In [5]:
url_codes = 'https://countrycode.org/'
data_codes = requests.get(url_codes).text
soup_codes = BeautifulSoup(data_codes, 'html.parser')    

In [6]:
#[tag.find_all("th") for tag in soup_codes.find_all('table')][1]

In [7]:
def procesaFila(tag):
    m = tag.find_all("td")
    return {
        "pais":m[0].find("a").text.strip(),
        #"cod_n":m[1].text,
        "Country":m[2].text.split(' /')[0]
    }
    
table_codes = soup_codes.find_all('table')[1]
table_codes_clean = [procesaFila(fila) for fila in table_codes.find_all("tr")[1:]]

In [8]:
countries = pd.DataFrame(table_codes_clean)

Hacemos el merge de los dos dataframes

In [9]:
result = pd.merge(ds, countries, how='left', on=['Country', 'Country'])
result.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,pais
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51,Andorra
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42,United Arab Emirates
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39,United Arab Emirates
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48,United Arab Emirates
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51,United Arab Emirates


In [13]:
result=result.drop(['Ownership Type','State/Province','Country','Postcode','Phone Number','Timezone'], axis=1)

In [14]:
result.to_csv('../input/starbucks_updt.csv', index = None, header=True)